In [1]:
import optuna

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
X.tail(4)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
568,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,0.05884,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


In [6]:
y.value_counts() / len(y)

0    0.627417
1    0.372583
dtype: float64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

In [8]:
X_train.shape, X_test.shape

((398, 30), (171, 30))

In [9]:
def objective(trial):

    rf_n_estimators = trial.suggest_int("rf_n_estimators", 100, 1000)
    rf_criterion = trial.suggest_categorical("rf_criterion", ['gini', 'entropy'])
    rf_max_depth = trial.suggest_int("rf_max_depth", 1, 4)
    rf_min_samples_split = trial.suggest_float("rf_min_samples_split", 0.01, 1)
    
    model = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        criterion=rf_criterion,
        max_depth=rf_max_depth,
        min_samples_split=rf_min_samples_split,
    )

    score = cross_val_score(model, X_train, y_train, cv=3)
    accuracy = score.mean()
    return accuracy

Randomized Search

In [10]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)


study.optimize(objective, n_trials=5)

[I 2021-07-22 20:32:17,131] Trial 0 finished with value: 0.9196286169970379 and parameters: {'rf_n_estimators': 856, 'rf_criterion': 'gini', 'rf_max_depth': 4, 'rf_min_samples_split': 0.44675074892735467}. Best is trial 0 with value: 0.9196286169970379.
[I 2021-07-22 20:32:19,702] Trial 1 finished with value: 0.8719146350725299 and parameters: {'rf_n_estimators': 846, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.6425949481900544}. Best is trial 0 with value: 0.9196286169970379.
[I 2021-07-22 20:32:23,221] Trial 2 finished with value: 0.9196476038581302 and parameters: {'rf_n_estimators': 997, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_min_samples_split': 0.5304332596733864}. Best is trial 2 with value: 0.9196476038581302.
[I 2021-07-22 20:32:25,027] Trial 3 finished with value: 0.9221538695222905 and parameters: {'rf_n_estimators': 503, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.4626601072693956}. Best is trial 3 with value

In [11]:
study.best_params

{'rf_n_estimators': 234,
 'rf_criterion': 'gini',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.060269386839397615}

In [12]:
study.best_value

0.9447482342219184

In [13]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_rf_criterion,params_rf_max_depth,params_rf_min_samples_split,params_rf_n_estimators,state
0,0,0.919629,2021-07-22 20:32:14.267564,2021-07-22 20:32:17.131278,00:00:02.863714,gini,4,0.446751,856,COMPLETE
1,1,0.871915,2021-07-22 20:32:17.131278,2021-07-22 20:32:19.702099,00:00:02.570821,entropy,3,0.642595,846,COMPLETE
2,2,0.919648,2021-07-22 20:32:19.717721,2021-07-22 20:32:23.221136,00:00:03.503415,entropy,1,0.530433,997,COMPLETE
3,3,0.922154,2021-07-22 20:32:23.223130,2021-07-22 20:32:25.027032,00:00:01.803902,entropy,3,0.462660,503,COMPLETE
4,4,0.944748,2021-07-22 20:32:25.027032,2021-07-22 20:32:25.851460,00:00:00.824428,gini,2,0.060269,234,COMPLETE


In [14]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)


study.optimize(objective, n_trials=5)

[I 2021-07-22 20:32:26,764] Trial 0 finished with value: 0.9171413381939697 and parameters: {'rf_n_estimators': 188, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.5688586472485292}. Best is trial 0 with value: 0.9171413381939697.
[I 2021-07-22 20:32:29,593] Trial 1 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 923, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.8433676459830718}. Best is trial 0 with value: 0.9171413381939697.
[I 2021-07-22 20:32:30,435] Trial 2 finished with value: 0.9196286169970379 and parameters: {'rf_n_estimators': 230, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.5506328733422843}. Best is trial 2 with value: 0.9196286169970379.
[I 2021-07-22 20:32:33,128] Trial 3 finished with value: 0.9371534897850687 and parameters: {'rf_n_estimators': 680, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.2919659622437388}. Best is trial 3 with value: 0.937

In [15]:
study.best_params

{'rf_n_estimators': 680,
 'rf_criterion': 'entropy',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.2919659622437388}

In [16]:
study.best_value


0.9371534897850687

CMA-ES

In [17]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.CmaEsSampler(),
)

study.optimize(objective, n_trials=5)

[I 2021-07-22 20:32:36,327] Trial 0 finished with value: 0.9146160856687172 and parameters: {'rf_n_estimators': 477, 'rf_criterion': 'entropy', 'rf_max_depth': 4, 'rf_min_samples_split': 0.5433199826138736}. Best is trial 0 with value: 0.9146160856687172.
[W 2021-07-22 20:32:36,543] The parameter 'rf_criterion' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2021-07-22 20:32:38,572] Trial 1 finished with value: 0.93215994531784 and parameters: {'rf_n_estimators': 550, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.42410538355952626}. Best is trial 1 with value: 0.93215994531784.
[W 2021-07-22 20:32:38,572] The parameter 'rf_criterion' in trial#2 is sampled independently by using `RandomSampler` instead of `CmaE

In [18]:
study.best_params

{'rf_n_estimators': 550,
 'rf_criterion': 'entropy',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.38260734807997737}

In [19]:
study.best_value

0.9371724766461608